In [104]:
pacman::p_load(
    tidyverse,
    rhdf5,
    caret,
    doMC,
    glue,
    kknn,
    MLmetrics
)

Installing package into ‘/home/el/R/x86_64-pc-linux-gnu-library/3.4’
(as ‘lib’ is unspecified)
also installing the dependency ‘ROCR’


MLmetrics installed


In [11]:
wdir <- getwd()
if (basename(wdir) != 'cytof') {
    if (basename(dirname(wdir)) == 'cytof') {
        wdir <- dirname(wdir)
    }
}
print(glue('setting working directory to {wdir}'))
setwd(wdir)

setting working directory to /home/el/projects/2018_asbjorn_cd4/cytof


In [20]:
predmarkers <- read_csv('input_data/common_markers_for_pred.csv') %>%
    mutate(fcsmarker = str_replace_all(Antibody, '[ -]', '_'))

Parsed with column specification:
cols(
  Antibody = col_character()
)


In [27]:
read_dataset <- function(path, h5_path) {
  M <- h5read(path, h5_path)
  colnames(M) <- h5readAttributes(path, h5_path)$colnames
  M
}

In [12]:
all_samples  <- read_csv('input_data/renamed/sample_meta_full.csv')

Parsed with column specification:
cols(
  unique_name = col_character(),
  sample_group = col_character(),
  category = col_character(),
  donor = col_character(),
  biosource = col_character(),
  disease = col_character(),
  instrument = col_character(),
  date = col_date(format = ""),
  sample = col_character(),
  disease_status = col_character(),
  sample_category = col_character(),
  note = col_character(),
  path = col_character()
)


In [46]:
ced_tet_samples <- all_samples %>%
    filter(biosource == 'PBMC' & disease == 'Ced' & disease_status != 'Challenge') %>%
    filter(sample_category != 'full' & (is.na(note) | note != 'exclude_ac'))

In [47]:
Ms <- ced_tet_samples %>%
    mutate(h5path = glue('samples/{unique_name}')) %>%
    .$h5path %>%
    map(partial(read_dataset, 'out/cytof.h5')) %>%
    map(~ .x[, predmarkers$fcsmarker])

names(Ms) <- ced_tet_samples$unique_name

In [99]:
num_rows <- Ms %>%
    map_int(nrow)
m <- with(ced_tet_samples, 
    tibble(sample = rep(unique_name, num_rows),
             sample_category = rep(sample_category, num_rows))) %>%
    mutate(sample_category = if_else(sample_category == 'tetramer+', 'tetramer_pos', 'tetramer_neg'))

In [100]:
M <- do.call(rbind, Ms)
M <- data.frame(M)
M$tetramer <- fct_relevel(m$sample_category, 'tetramer_neg')

In [101]:
set.seed(1234)
trainingRows <- createDataPartition(m$sample_category, p = .8, list = FALSE)
traindf <- M %>% slice(trainingRows)
tesdft <- M %>% slice(-trainingRows)

In [102]:
# can also include  summaryFunction = multiClassSummary 
# possible validation methods are method= "cv", "boot632", "LGOCV","LOOCV","repeatedcv", "boot" 
set.seed(12345)
control <- trainControl(method="repeatedcv", number=10, repeats=3, classProbs= TRUE, summaryFunction = multiClassSummary)

# can be "Accuracy",   "logLoss", "ROC",   "Kappa"
metric <- "logLoss"

Register 6 cores for caret computation

In [89]:
registerDoMC(cores = 6)

In [108]:
pacman::p_load(kernlab)
SEED <- 1234

In [105]:
SEED <- 1234
m_kknn <- train(tetramer ~ ., data=traindf, method="kknn", metric=metric, 
       trControl=control, preProcess = c("center", "scale") )
print(m_kknn)

k-Nearest Neighbors 

28523 samples
   21 predictor
    2 classes: 'tetramer_neg', 'tetramer_pos' 

Pre-processing: centered (21), scaled (21) 
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 25671, 25670, 25671, 25671, 25670, 25670, ... 
Resampling results across tuning parameters:

  kmax  logLoss    AUC        prAUC      Accuracy   Kappa      F1       
  5     0.2195783  0.9375985  0.1972959  0.9878811  0.8300007  0.9937075
  7     0.2097369  0.9394230  0.2170298  0.9880096  0.8318535  0.9937742
  9     0.1968051  0.9416457  0.2692257  0.9883952  0.8365946  0.9939754
  Sensitivity  Specificity  Pos_Pred_Value  Neg_Pred_Value  Precision
  0.9952596    0.8037559    0.9921618       0.8725901       0.9921618
  0.9953204    0.8055769    0.9922342       0.8742635       0.9922342
  0.9956851    0.8064887    0.9922730       0.8830515       0.9922730
  Recall     Detection_Rate  Balanced_Accuracy
  0.9952596  0.9569120       0.8995078        
  0.9953204  0.

In [ ]:
SEED <- 1234
m_rf <- train(tetramer ~ ., data=traindf, method="rf", metric=metric, 
       trControl=control, preProcess = c("center", "scale") )
print(m_rf)

In [ ]:
SEED <- 1234
m_svmLin1 <- train(tetramer ~ ., data=traindf, method="svmLinear", metric=metric, 
       trControl=control, preProcess = c("center", "scale") )
print(m_svmLin1)

In [ ]:
SEED <- 1234
m_svmLin2 <- train(tetramer ~ ., data=traindf, method="svmLinear2", metric=metric, 
       trControl=control, preProcess = c("center", "scale") )
print(m_svmLin2)

In [ ]:
SEED <- 1234
m_svmRadial <- train(tetramer ~ ., data=traindf, method="svmRadial", metric=metric, 
       trControl=control, preProcess = c("center", "scale") )
print(m_svmLin2)

In [106]:
123

[1] 123